In [31]:
from google.colab import files
uploaded = files.upload()

Saving house.zip to house.zip


In [32]:
!unzip house.zip

Archive:  house.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [33]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

california_data = pd.read_csv('sample_submission.csv')

In [34]:
print(train_data.shape)
print(test_data.shape)
train_data.head()

(1460, 81)
(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [35]:
for col in train_data.columns:
    if train_data[col].isnull().sum() > 0:
        if train_data[col].dtype == 'object':
            train_data[col] = train_data[col].fillna("None")
        else:
            train_data[col] = train_data[col].fillna(train_data[col].median())

In [36]:
cat_cols = train_data.select_dtypes(include=['object']).columns

train_data = pd.get_dummies(train_data, columns=cat_cols)
test_data = pd.get_dummies(test_data, columns=cat_cols)


In [37]:
X = train_data.drop(columns=['Id', 'SalePrice'])
y = train_data['SalePrice']

X_test = test_data.drop(columns=['Id'])


In [38]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_valid)

mae = mean_absolute_error(y_valid, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 17776.588732876713


In [41]:
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

In [42]:
print("X_train columns:", X_train.columns)
print("X_test columns:", X_test.columns)



X_train columns: Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=303)
X_test columns: Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=303)


In [43]:
if 'SalePrice' in X_test.columns:
    X_test = X_test.drop(columns=['SalePrice'])


In [44]:
train_cols = set(X_train.columns)
test_cols = set(X_test.columns)

print("Columns in train but not in test:", train_cols - test_cols)
print("Columns in test but not in train:", test_cols - train_cols)


Columns in train but not in test: set()
Columns in test but not in train: set()


In [45]:
y_test_pred = model.predict(X_test)

submission = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': y_test_pred})
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")


Submission file created successfully!


In [46]:
import joblib

joblib.dump(model, 'house_price_model.pkl')

print("🎉 โมเดลถูกบันทึกเป็น house_price_model.pkl")


🎉 โมเดลถูกบันทึกเป็น house_price_model.pkl


In [47]:
import pandas as pd
import joblib

model = joblib.load('house_price_model.pkl')

submission = pd.read_csv('submission.csv')

print(submission.head())


     Id  SalePrice
0  1461  131164.88
1  1462  156002.00
2  1463  177312.34
3  1464  188278.90
4  1465  205146.50


In [48]:
print("จำนวนฟีเจอร์ที่โมเดลต้องการ:", model.n_features_in_)
print("ชื่อฟีเจอร์ที่ใช้ฝึก:", X_train.columns.tolist())


จำนวนฟีเจอร์ที่โมเดลต้องการ: 303
ชื่อฟีเจอร์ที่ใช้ฝึก: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_None', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub', 'Utilities_NoSeWa', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 

In [50]:
import pandas as pd
import numpy as np

input_data = pd.DataFrame(np.zeros((1, X_train.shape[1])), columns=X_train.columns)

input_data.loc[0, 'GrLivArea'] = 1000   # พื้นที่ใช้สอย (ตร.ฟุต)
input_data.loc[0, 'BedroomAbvGr'] = 1   # จำนวนห้องนอน
input_data.loc[0, 'FullBath'] = 1       # จำนวนห้องน้ำ

predicted_price = model.predict(input_data)[0]

print(f"🏡 ราคาบ้านที่คาดการณ์: ${predicted_price:,.2f}")


🏡 ราคาบ้านที่คาดการณ์: $71,224.00
